In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from pandas_profiling import ProfileReport

# Semana 1

## 1. Importação dos Dados
* Dados importados a partir de um arquivo CSV e armazenados em uma estrutura Dataframe do Pandas

In [59]:
df_dados = pd.read_csv('Banco_de_dados_bancosaudemental.csv',encoding="ISO-8859-1",delimiter=';')

In [3]:
df_dados.head()

,SAMPLEID,SC1,SC1_1,SC2,SC2A,SC3,SC3A,SC7,SC8_1,SC8_2,...,notreat12_notsevere,notreat12_anypredisposing,notreat12_service,qids_none,qids_mild,qids_mod,qids_sev,qids_vsev,qids_sevverysev,_merge
0,3201000001-01,32,1,4,4.0,5,1.0,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
1,3201000002-01,31,1,8,4.0,5,1.0,3,3,3,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
2,3201000003-01,60,1,9,4.0,1,NaN,2,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,matched (3)
3,3201000004-01,34,1,5,4.0,5,1.0,3,3,2,...,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
4,3201000005-01,84,2,41,4.0,4,5.0,3,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)


## 2. Selecionando os Atributos Relacionados à Depressão
* Na base de dados do São Paulo Megacity os Atributos relacionados a Depressão começam com a letra "D", com 3 exceções que são tratadas no código a seguir, tais atributos são selecionados e armazenados em uma lista

In [4]:
lista=[]
for x in df_dados.columns:
  if x[0] == "D":
    lista.append(x)
    
    
#Marcadores de Diagnóstico

# Major Depressive Disorder (MDD):
lista.extend(['dsm_mddh']) #'d_mddh12','d_mddh30'

# Major Depressive Episode (MDE):
#lista.extend(['d_mde12', 'd_mde30','dsm_mde'])


# Minor Depressive Disorder (MND):
#lista.extend(['d_mndh12', 'd_mndh30', 'dsm_mndh','d_mnd12','d_mnd30','dsm_mnd'])

#Recurrent Brief Depression Disorder (RBD):
#lista.append('dsm_rbd')


#Removendo Atributos iniciados com "D" não relacionados a Depressão:
lista = [x for x in lista if not x == 'D26_YESC' or x == 'D0' or x == 'D29C1']

## 3. Criando novo Dataframe a partir dos Atributos Selecionados
* Utilizando a lista de atributos relacionados à depressão é criado um novo dataframe somente com tais variáveis

In [5]:
df_depression = df_dados[lista]

In [6]:
df_depression

,D0,D1,D1A,D1B,D2,D9,D9A1,D9A2,D10,D11,...,D84A,D84B,D84C,D86,D87,D87A,D87_1,D88,D89,dsm_mddh
0,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,NaN,5
1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,5
2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.0,1.0,NaN,1.0,1.0,59.0,2.0,4.0,NaN,1
3,1.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,NaN,5
4,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.0,NaN,5
5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


## 4. Análise e Tratamento de Valores Nulos
* Colunas com alto indíce de valores nulos serão retiradas nesta etapa

In [7]:
excluir_nulos = []
nulos = df_depression.isnull().sum()
nulos.sort_values(ascending=False,inplace=True)
for i in nulos.index:
    print(i,"--->",nulos[i])
    if nulos[i] >= 5037*0.8:
        excluir_nulos.append(i)
        

D23A15 ---> 5037
D23A09 ---> 5037
D23A17 ---> 5037
D23A16 ---> 5037
D23A14 ---> 5037
D23A13 ---> 5037
D23A12 ---> 5037
D23A11 ---> 5037
D23A10 ---> 5037
D23A18 ---> 5037
D23A07 ---> 5036
D23A08 ---> 5036
D50 ---> 5035
D45 ---> 5034
D49 ---> 5033
D23A06 ---> 5033
D48 ---> 5033
D47 ---> 5033
D43 ---> 5031
D46 ---> 5031
D42_1 ---> 5031
D42 ---> 5031
D44 ---> 5031
D23A05 ---> 5030
D41 ---> 5026
D11 ---> 5022
D23A04 ---> 5010
D9A2 ---> 5008
D10 ---> 5007
D9A1 ---> 5007
D89 ---> 5000
D9 ---> 4992
D37B ---> 4973
D38C ---> 4959
D23A03 ---> 4959
D87A ---> 4945
D38A_9 ---> 4939
D15 ---> 4939
D29C1 ---> 4925
D58 ---> 4887
D38A_7B ---> 4881
D61 ---> 4877
D60 ---> 4863
D59A ---> 4863
D59 ---> 4863
D22D1 ---> 4859
D84C ---> 4849
D22D ---> 4837
D22C ---> 4837
D38A_7A ---> 4835
D38A_8 ---> 4835
D14A ---> 4828
D38A_6 ---> 4825
D26D1 ---> 4819
D26D ---> 4818
D26O ---> 4808
D29B ---> 4805
D23A02 ---> 4803
D39B ---> 4776
D56 ---> 4756
D57 ---> 4756
D37M_2 ---> 4751
D37N_2 ---> 4750
D26H ---> 4745
D54 --->

* ###  Exclusão das colunas com 80% ou mais de valores nulos

In [8]:
df_depression.drop(excluir_nulos,axis=1,inplace=True)

/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
variaveis_count = df_depression.count().sort_values(ascending = False)

In [10]:
lista = []
for i in range(len(df_depression)):
    x = (len(df_depression.iloc[i].dropna())/len(variaveis_count))*100
    if x >= 50:  # Porcentagem das colunas sem valor nulo para cada entrada
        lista.append(True)
    else:
        lista.append(False)

* ###  Serão utilizadas somente linhas com 65% ou mais de variáveis não nulas

In [11]:
# "df_depression_65" contém as entradas do "df_depression" com 65% das colunas sem valor nulo
df_depression_65 = df_depression.iloc[lista]

In [12]:
len(df_depression_65)

1212

## 5. Pandas Profiling (df_depression_65)
* Geração de relatório 

In [13]:
#pfl_df_depression65 = ProfileReport(df_depression_65, title='Pandas Profiling Report', html={'style':{'full_width':False}},minimal=True)
#pfl_df_depression65.to_file(output_file="pfl_df_depression65.html")

## 6. Métricas e Correlações
* Tabela de correlação e função describe() do Dataframe

In [14]:
def verifica_maiores_correlacoes_absolutas(dados,correlacao=0.80):
    #objeto recebido e do tipo pandas.dataframe()
    #correlacao deve ser do tipo float
    #padrao verificado para correlacoes e acima de 80%
    if correlacao>=1:
        correlacao=(correlacao/100.0)
    corr = dados.corr()
    lista = []
    for coluna in corr.columns:
        for linha in corr.index:
            if (coluna != linha) and (abs(corr[coluna][linha]) >= correlacao):
                #print(coluna,' possui com  ',linha,' Correlacao de -->',corr[coluna][linha])
                dict = {'coluna':coluna,'linha':linha,'corr':corr[coluna][linha]}
                lista.append(dict)
                #print("%s  Vs   %s \t\t || Correlacao de --> %f ||"%(coluna,linha,corr[coluna][linha]))
    return lista

In [15]:
df_depression_65.describe()

,D0,D1,D1A,D12,D12A,D13,D16,D17,D18,D19,...,D29A,D29D,D37D,D37E,D37G,D38,D72,D87_1,D88,dsm_mddh
count,1212.000000,1165.00000,1109.000000,1207.000000,68.000000,68.0,1143.000000,1212.000000,1212.000000,1212.000000,...,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1212.000000,1212.000000,1212.000000
mean,1.042904,1.20000,1.680794,1.227838,21.735294,1.0,3.622922,2.962871,1.819307,2.231848,...,4.247312,1.542597,2.659222,1.246485,1.542597,2.973532,3.395368,105.429868,4.281353,2.082508
std,0.222160,0.90265,1.510194,0.936785,101.840322,0.0,0.883037,0.921049,0.972550,1.102452,...,1.597041,0.498388,1.131868,0.431142,0.498388,2.029002,1.968084,305.507382,2.587376,1.777869
min,1.000000,1.00000,1.000000,1.000000,3.000000,1.0,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,1.000000,1.00000,1.000000,1.000000,3.000000,1.0,3.000000,2.000000,1.000000,1.000000,...,5.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000
50%,1.000000,1.00000,1.000000,1.000000,7.000000,1.0,4.000000,3.000000,2.000000,2.000000,...,5.000000,2.000000,3.000000,1.000000,2.000000,1.000000,5.000000,1.000000,4.000000,1.000000
75%,1.000000,1.00000,1.000000,1.000000,7.000000,1.0,4.000000,4.000000,2.000000,3.000000,...,5.000000,2.000000,3.000000,1.000000,2.000000,5.000000,5.000000,2.000000,6.000000,5.000000
max,3.000000,8.00000,8.000000,8.000000,830.000000,1.0,8.000000,9.000000,9.000000,9.000000,...,8.000000,2.000000,9.000000,2.000000,2.000000,9.000000,9.000000,999.000000,15.000000,5.000000


In [16]:
lista_corr = verifica_maiores_correlacoes_absolutas(df_depression_65,correlacao=0.3)

In [17]:
lista_corr_2 = []
for i in lista_corr:
    if i['coluna'] == "dsm_mddh":
        lista_corr_2.append(i)
        
        
        
for i in lista_corr_2:
    print("%s  Vs   %s \t\t || Correlacao de --> %f ||"%(i['coluna'],i['linha'],i['corr']))
    print('')


dsm_mddh  Vs   D26J 		 || Correlacao de --> 0.320987 ||

dsm_mddh  Vs   D26R 		 || Correlacao de --> 0.340269 ||



## 7. Carregamento do Dicionário de Variáveis
* O carregamento é feito a partir de um arquivo CSV e armazenado em uma estrutura Dataframe do pandas, logo após é impresso as definições de cada variável

In [18]:
df_variaveis_depre_temp = pd.read_csv('/home/gabriel/Desktop/CIDI Depressão - Mapeamento de variáveis _2_CSV.csv',delimiter=';',encoding='ISO-8859-1')

In [19]:
df_variaveis_depre_temp = df_variaveis_depre_temp.rename(columns={'Pergunta/Instrução':'Pergunta_Instrucao','Página':'Pagina','Variável na base de dados': 'Variavel','Possíveis valores':'Possiveis_valores'})

In [20]:
df_variaveis_depre_temp = df_variaveis_depre_temp[(df_variaveis_depre_temp.Variavel != 'Not Found') & (df_variaveis_depre_temp.Variavel.notna())]

In [21]:
Variaveis_D = df_variaveis_depre_temp['Variavel'].str.split(',')

In [22]:
df_variaveis_depre_v2 = pd.DataFrame(columns=df_variaveis_depre_temp.columns)

In [23]:
df_variaveis_depre_v2 = df_variaveis_depre_v2[['Variavel','Pergunta_Instrucao', 'Possiveis_valores']]

In [24]:
for i in Variaveis_D.index:
    for variavel in Variaveis_D[i]:
        df_variaveis_depre_v2.loc[len(df_variaveis_depre_v2)] = [variavel.strip(),df_variaveis_depre_temp.loc[i].Pergunta_Instrucao,df_variaveis_depre_temp.loc[i].Possiveis_valores]

In [25]:
dict_variaveis_depre_v2 = {}
for row in df_variaveis_depre_v2.itertuples():
    dict_variaveis_depre_v2[row.Variavel] = row.Pergunta_Instrucao

In [26]:
dict_valores_depre_v2 = {}
for row in df_variaveis_depre_v2.itertuples():
    dict_valores_depre_v2[row.Variavel] = row.Possiveis_valores

In [27]:
for i in dict_variaveis_depre_v2:
    print('%s ---> %s\n\n'%(i,dict_variaveis_depre_v2[i]))

D1 ---> O(A) Sr(a). mencionou anteriormente que teve períodos de vários dias ou mais em que se sentia triste, vazio(a) oudeprimido(a) a maior parte do dia. Durante fases como essa, o(a) Sr(a). alguma vez se sentiu 
desanimado(a) em relação a como as coisas andavam em sua vida?


D1A ---> Durante as fase sem que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria dascoisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D1B ---> Durante as fases em que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria das coisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D2 ---> (O Sr/a Sra.) mencionou anteriormente queteve períodos de vários dias ou mais em que se sentia desanimado(a)em relação a como as coisas andavam em sua vida. Durante fases como essa, o(a) Sr(a). alguma vez 
perdeu o interesse na maioria das coisas como trabalho, diversão e outras a

# Semana 2

## 1. Balanceamento dos Dados
* Os dados são balanceados a partir da variável dependente, são igualados os indivíduos com diagnóstico positivo e negativo para depressão

In [28]:
count_5 = len(df_depression_65[df_depression_65.dsm_mddh == 5]) # Diagnóstico Positivo
count_5

328

In [29]:
count_1 = len(df_depression_65[df_depression_65.dsm_mddh == 1]) # Diagnóstico Negativo
count_1

884

In [30]:
df_dep_65_bal = df_depression_65[df_depression_65.dsm_mddh == 1].sample(count_5,replace=True,random_state = 5)
df_dep_65_bal = pd.concat([df_dep_65_bal,df_depression_65[df_depression_65.dsm_mddh == 5]],axis=0)

In [31]:
# Dataframe Balanceado
df_dep_65_bal.head()

,D0,D1,D1A,D12,D12A,D13,D16,D17,D18,D19,...,D29A,D29D,D37D,D37E,D37G,D38,D72,D87_1,D88,dsm_mddh
4962,1.0,5.0,NaN,1.0,NaN,NaN,4.0,2.0,2.0,2.0,...,5.0,1.0,2.0,1.0,1.0,5.0,5.0,0.0,4.0,1
1207,1.0,1.0,5.0,1.0,NaN,NaN,8.0,4.0,1.0,1.0,...,5.0,1.0,2.0,1.0,1.0,5.0,1.0,1.0,6.0,1
4072,1.0,1.0,1.0,1.0,NaN,NaN,4.0,2.0,3.0,2.0,...,1.0,2.0,1.0,2.0,2.0,1.0,1.0,4.0,4.0,1
714,1.0,1.0,1.0,1.0,NaN,NaN,3.0,4.0,1.0,1.0,...,5.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,2.0,1
2373,1.0,1.0,1.0,5.0,3.0,1.0,NaN,1.0,3.0,2.0,...,5.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,5.0,1


## 2. Definindo Conjuntos de Treino e Teste
* O dataframe será dividido em um conjunto de treino e outro conjunto de teste. O conjunto de treino possui 80% do volume das linhas enquanto o conjunto de teste possui os 20% restantes.

In [32]:
X_train,X_test,y_train,y_test = train_test_split(df_dep_65_bal,df_dep_65_bal['dsm_mddh'],train_size = 0.8,test_size = 0.2,random_state = 5)

In [33]:
print(len(X_train))
print(len(X_test))

524
132


## 3. Selecionando Principais Atributos com Chi2
* O teste do chi2 irá selecionar os N melhores atributos para serem utilizados no modelo estatístico

In [34]:
X_train.fillna(0,inplace=True)
Melhores_Atributos = SelectKBest(chi2,k=50).fit(X_train.drop('dsm_mddh',axis=1),y_train)
Lista_Melhores = list(X_train.drop('dsm_mddh',axis=1).columns[Melhores_Atributos.get_support()])

/home/gabriel/.local/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [35]:
X_train.drop('dsm_mddh',axis=1).columns[Melhores_Atributos.get_support()]

Index(['D1', 'D1A', 'D12A', 'D16', 'D17', 'D18', 'D19', 'D22', 'D22A', 'D22B',
       'D22B1', 'D23', 'D24A', 'D24B', 'D24C', 'D24D', 'D24E', 'D24F', 'D26A',
       'D26G', 'D26I', 'D26J', 'D26L', 'D26P', 'D26R', 'D26S', 'D26T', 'D26U',
       'D26W', 'D26X', 'D26Y', 'D26Z', 'D26AA', 'D26BB', 'D26CC', 'D26FF',
       'D26GG', 'D26HH', 'D26II', 'D27A', 'D28', 'D29', 'D29A', 'D29D', 'D37E',
       'D37G', 'D38', 'D72', 'D87_1', 'D88'],
      dtype='object')

In [36]:
Score_Melhores = pd.Series(Melhores_Atributos.scores_)
Score_Melhores.index = X_train.drop('dsm_mddh',axis=1).columns
Score_Melhores = Score_Melhores.sort_values(ascending=False)
lst_melhores = []
#for i in Score_Melhores.index:
    #if Score_Melhores[i] > 5:
        #print('%s --->  %f'%(i,Score_Melhores[i]))
    #lst_melhores.append(i)

In [37]:
for i in Score_Melhores.index:
    print('%s --->  %f'%(i,Score_Melhores[i]))

D22B --->  16495.307904
D87_1 --->  4814.453801
D12A --->  864.169532
D22A --->  245.809761
D26R --->  110.114344
D26J --->  69.830424
D26AA --->  53.529660
D26P --->  50.667628
D24F --->  50.222632
D26II --->  45.617181
D26GG --->  44.883361
D26S --->  43.745335
D29 --->  42.614284
D26L --->  41.709874
D26BB --->  40.462918
D26U --->  39.414891
D24E --->  38.679249
D26T --->  37.472129
D24B --->  33.901645
D26HH --->  32.218377
D26A --->  28.642468
D22B1 --->  27.223344
D26G --->  20.819246
D26FF --->  19.968025
D26Y --->  17.962925
D22 --->  13.031003
D26CC --->  11.483513
D24D --->  10.655286
D1A --->  10.090126
D88 --->  9.765314
D72 --->  8.496179
D24C --->  7.529713
D19 --->  7.047495
D26Z --->  6.612669
D28 --->  6.395312
D23 --->  6.051334
D26X --->  5.978186
D26W --->  5.761530
D1 --->  5.451750
D18 --->  4.994345
D38 --->  3.328114
D17 --->  2.674475
D29A --->  2.475685
D16 --->  1.838172
D29D --->  0.874549
D37G --->  0.874549
D37E --->  0.747963
D26I --->  0.395619
D27A ---

In [38]:
lst_melhores = Lista_Melhores

In [39]:
for i in dict_variaveis_depre_v2:
    if i in lst_melhores:
        print('%s ---> %s\n\n'%(i,dict_variaveis_depre_v2[i]))

D1 ---> O(A) Sr(a). mencionou anteriormente que teve períodos de vários dias ou mais em que se sentia triste, vazio(a) oudeprimido(a) a maior parte do dia. Durante fases como essa, o(a) Sr(a). alguma vez se sentiu 
desanimado(a) em relação a como as coisas andavam em sua vida?


D1A ---> Durante as fase sem que se sentia triste, vazio(a) ou deprimido(a), o(a) Sr(a). alguma vez perdeu o interessena maioria dascoisas como trabalho, diversão e outras atividades de que habitualmente gostava?


D12A ---> Quantos dias durou o período mais longo em que o(a) Sr(a). se sentiu (FRASE-CHAVE) durante a maior parte do dia?


D16 ---> Pense nas fases de duas semanas ou mais em que (esse problema/esses problemas) com seu estado de ânimo(foi/foram) mais intenso(s) e freqüente(s). Nessas fases, o(a) Sr(a). geralmente se sentia (FRASE-CHAVE) 
durante---menos de 1 hora por dia, entre 1 e 3 horas, entre 3 e 5 horas ou mais de 5 horas por dia?
MENOS DE 1 HORA...................................1
ENTRE 1 E 3

In [40]:
lst_melhores.append('dsm_mddh')
X_train = X_train[lst_melhores]
X_test = X_test[lst_melhores]

In [41]:
len(lst_melhores)

51

# Semana 3
# Premissas da Regressão Logística

## 1. Relação Linear entre Variável Dependente e Variáveis Independentes

* Uma relação linear é uma tendência nos dados que pode ser modelada por uma linha reta. Por exemplo, suponha que uma empresa aérea deseja estimar o impacto do preço dos combustíveis sobre o custo dos voos. Eles determinam que, para cada aumento de um dólar no preço de um galão de combustível, o custo do voo Los Angeles-New York aumenta em torno de US$ 3500. Isso descreve uma relação linear entre o custo do combustível de aviação e o custo do voo.


In [42]:
for i in lst_melhores:
    print(X_train[['dsm_mddh',i]].corr())
    print('')

          dsm_mddh        D1
dsm_mddh  1.000000  0.113517
D1        0.113517  1.000000

          dsm_mddh      D1A
dsm_mddh   1.00000  0.10988
D1A        0.10988  1.00000

          dsm_mddh     D12A
dsm_mddh   1.00000 -0.05478
D12A      -0.05478  1.00000

          dsm_mddh       D16
dsm_mddh  1.000000 -0.078075
D16      -0.078075  1.000000

          dsm_mddh       D17
dsm_mddh  1.000000 -0.119268
D17      -0.119268  1.000000

          dsm_mddh       D18
dsm_mddh  1.000000  0.127526
D18       0.127526  1.000000

          dsm_mddh       D19
dsm_mddh  1.000000  0.153047
D19       0.153047  1.000000

          dsm_mddh       D22
dsm_mddh  1.000000  0.125113
D22       0.125113  1.000000

          dsm_mddh      D22A
dsm_mddh  1.000000  0.050671
D22A      0.050671  1.000000

          dsm_mddh      D22B
dsm_mddh  1.000000  0.194296
D22B      0.194296  1.000000

          dsm_mddh     D22B1
dsm_mddh  1.000000 -0.226983
D22B1    -0.226983  1.000000

          dsm_mddh       D23
dsm_mddh 

In [43]:
#X_train.drop(['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1'],axis=1,inplace=True)
#X_test.drop(['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1'],axis=1,inplace=True)

In [44]:
#for i in ['D22C', 'D64A', 'D87_1', 'D64C', 'D59', 'D39A', 'D38A_1','D29' ,'D38B','D59A','D68', 'D54', 'D37H_1']:
#    lst_melhores.remove(i)

In [45]:
'''lst_exclusao = []
for i in lst_melhores:
    corr_dsm = X_train[['dsm_mddh',i]].corr()
    print(corr_dsm[i][0])'''

"lst_exclusao = []\nfor i in lst_melhores:\n    corr_dsm = X_train[['dsm_mddh',i]].corr()\n    print(corr_dsm[i][0])"

### Observações:
* As Variáveis "D64A", "D64C", "D84A", "D59", "D39A", "D38A_1", "D29", "D38B", "D59A","D68", "D54" e "D37H_1" possuem Relação Linear (Pearson) abaixo/igual a 0.2 com a Variável Dependente, portanto não serão utilizadas no Modelo

* A Variável "D22C" não será utilizada por estar nula.

## 2. Ausência de Heterocedasticidade

* Fenômeno onde a variância dos erros de um modelo matemático não é igual para cada observação

## 3. Ausência de Multicolinearidade

* Multicolinearidade em regressão é uma condição que ocorre quando algumas variáveis preditoras no modelo estão correlacionadas a outras variáveis preditoras. A multicolinearidade forte é problemática porque pode aumentar a variância dos coeficientes de regressão, tornando-os instáveis.


In [46]:
X_train.drop(['dsm_mddh'],axis=1).corr()

,D1,D1A,D12A,D16,D17,D18,D19,D22,D22A,D22B,...,D28,D29,D29A,D29D,D37E,D37G,D38,D72,D87_1,D88
D1,1.000000,-0.187549,-0.011808,-0.015328,0.031743,0.104561,0.030594,0.074254,-0.021425,-0.038316,...,-0.128677,-0.060089,0.056452,-0.068045,-0.107854,-0.068045,0.033133,0.117109,0.143702,0.144522
D1A,-0.187549,1.000000,-0.008089,-0.014836,-0.161937,0.068386,0.140514,0.057856,-0.051527,0.029338,...,-0.116411,-0.044172,0.030827,-0.023653,-0.019299,-0.023653,0.103387,0.073059,0.043508,0.164814
D12A,-0.011808,-0.008089,1.000000,-0.163206,0.043115,-0.028845,0.063966,-0.027640,-0.007474,-0.012478,...,0.047188,-0.020971,0.024364,0.049604,-0.018004,0.049604,-0.038223,-0.045848,-0.023437,-0.006795
D16,-0.015328,-0.014836,-0.163206,1.000000,0.209889,-0.214745,-0.204706,-0.081386,0.034694,0.067770,...,0.136963,0.024574,0.043509,-0.036702,-0.016267,-0.036702,-0.009297,-0.122573,-0.029484,-0.022010
D17,0.031743,-0.161937,0.043115,0.209889,1.000000,-0.168829,-0.147452,-0.096868,0.030731,0.040423,...,0.155961,-0.008615,-0.034127,0.038177,-0.027951,0.038177,-0.022764,-0.068949,0.001552,-0.113862
D18,0.104561,0.068386,-0.028845,-0.214745,-0.168829,1.000000,0.467359,0.055206,-0.059747,-0.042336,...,-0.249606,-0.075279,0.119648,-0.109367,-0.108931,-0.109367,0.018361,0.131205,0.005621,0.116408
D19,0.030594,0.140514,0.063966,-0.204706,-0.147452,0.467359,1.000000,0.049532,0.069870,-0.039256,...,-0.339619,-0.061544,0.059775,0.037400,-0.035139,0.037400,0.024776,0.161417,-0.065691,0.150761
D22,0.074254,0.057856,-0.027640,-0.081386,-0.096868,0.055206,0.049532,1.000000,-0.194899,-0.102555,...,-0.199259,0.083328,0.124661,-0.034310,-0.024503,-0.034310,0.020926,0.106742,0.099948,0.149423
D22A,-0.021425,-0.051527,-0.007474,0.034694,0.030731,-0.059747,0.069870,-0.194899,1.000000,0.001887,...,0.038060,0.038417,-0.068227,0.020790,0.066877,0.020790,0.016388,0.040590,0.115764,-0.035094
D22B,-0.038316,0.029338,-0.012478,0.067770,0.040423,-0.042336,-0.039256,-0.102555,0.001887,1.000000,...,0.088603,0.031748,0.004157,0.016567,-0.023370,0.016567,0.001640,-0.007226,0.087097,-0.006216


* ####  D61  Vs. D56
* #### D72A Vs. D87A
* #### D72A Vs. D84C
* #### D87A Vs. D84C
* #### D26F Vs. D84C
* #### D37I_1 Vs. D84C

In [47]:
#X_train.drop(['D84C','D61','D87A'],axis=1,inplace=True)
#X_test.drop(['D84C','D61','D87A'],axis=1,inplace=True)

## 4. Resíduos

* Os resíduos indicam a variação natural dos dados, um fator aleatório (ou não) que o modelo não capturou. Se as pressuposições do modelo são violadas, a análise será levada a resultados duvidosos e não confiáveis para inferência. Essas falhas do modelo nos pressupostos podem ser oriundas de diversos fatores como não linearidade, não-normalidade, heterocedasticidade, não-independência e isso pode ser causado por pontos atípicos (observações discrepantes), que podem influenciar, ou não, no ajuste do modelo.


## 5. Regressão Logística

In [48]:
X_train.fillna(0,inplace=True)

In [49]:
X_test.fillna(0,inplace=True)

In [50]:
modelo = LogisticRegression(random_state=5,max_iter=1000000,C=3).fit(X_train.drop(['dsm_mddh'],axis=1),y_train)

In [51]:
score_treino = modelo.score(X_train.drop(['dsm_mddh'],axis=1),y_train)

In [52]:
print('Score de Acuracia [TREINO]:',score_treino)

Score de Acuracia [TREINO]: 0.7862595419847328


In [53]:
score_teste = modelo.score(X_test.drop(['dsm_mddh'],axis=1),y_test)

In [54]:
print('Score de Acuracia [TESTE]:',score_teste)

Score de Acuracia [TESTE]: 0.7727272727272727


## Semana 4

## 1.  Avaliação do Modelo

### 1.1 Matriz de Confusão
* Matriz que permite visualizar o desempenho do modelo

In [55]:
y_pred = modelo.predict(X_test.drop(['dsm_mddh'],axis=1))
matriz_confusao = confusion_matrix(y_test,y_pred)
print(matriz_confusao)

[[55  8]
 [22 47]]


In [56]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.71      0.87      0.79        63
           5       0.85      0.68      0.76        69

    accuracy                           0.77       132
   macro avg       0.78      0.78      0.77       132
weighted avg       0.79      0.77      0.77       132



![Precision](https://i.imgur.com/lBX6xb0.png)
![Recall](https://i.imgur.com/auzyA0v.png)
![F1-Score](https://i.imgur.com/uRnhSGP.png)


* #### f1-score = Média harmônica de Precision e Recall